# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$.
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки.
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вершину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии.

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

import random
from pprint import pprint

Код для расчёта энтропии:

In [ ]:
def entropy(y):

    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))

    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер.

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения узлов дерева (используйте, например, `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признаков и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три разных критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики доля правильных ответов (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
iris = datasets.load_iris()
df = pd.DataFrame({'type': iris.target,
                   'sepal length': iris.data[:,0],
                   'sepal width': iris.data[:,1],
                   'petal length': iris.data[:,2],
                   'petal width': iris.data[:,3]})

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns='type',axis=1), df.type)
x_train.insert(0,'type',y_train)
x_train

In [ ]:
from numpy.core.fromnumeric import shape
def findIG(l,r):#находит функционал качества для данного разделения
  llen=len(l)
  rlen=len(r)
  lrlen=llen+rlen
  lr=np.hstack((l,r))
  #print(entropy(lr),entropy(l),entropy(r))
  return (entropy(lr)-(llen/lrlen*entropy(l)+rlen/lrlen*entropy(r)))
  #return (entropy(lr)-(entropy(l)+entropy(r)))

def splitTree(df,numParts):#выбираем как лучше разделить наши данные на
  feachure,maxIG,sep = 0,0,0 #на данное количество частей
  L=pd.DataFrame()
  R=pd.DataFrame()
  for i in range(1,df.shape[1]):
    col = df.columns[i]
    x = np.linspace(min(df[col]),max(df[col]),numParts)
    for j in x:
      l=df.iloc[np.where(df[col]<j)]
      r=df.iloc[np.where(df[col]>=j)]
      if (findIG(l.iloc[:,0],r.iloc[:,0])>maxIG):
        maxIG = findIG(l.iloc[:,0],r.iloc[:,0])
        feachure = i
        sep = j
        L=l.copy()
        R=r.copy()
  return(feachure,sep,L,R)

def isStop(deep,listObj,listCount,stopcrit):#проверка нужных критериев остановки
  #print(stopcrit)
  if stopcrit[0]==1 and deep==5:
    return True
  if stopcrit[1]==1 and listObj<=5:
    return True
  if stopcrit[2]==1 and listCount==5:
    return True
  return False

def maxProb(x):#выбирает самое частое значение
  return x.value_counts().index[0]

def makeTree(cur,tree,deep,df,listCount,numParts,stopcrit=[0,0,0]):# создает дерево рекурсивно
  #print(tree.shape[0],tree)
  if (not(isStop(deep,df.shape[0],listCount,stopcrit)) and entropy(df.iloc[:,0]))!=0:
    #проверяем условия остановки
    feach,sep,l,r=splitTree(df,numParts)# поделили дерево

    tree[cur,0]=feach
    tree[cur,1]=sep# заполнели информацию о делении в данную ветку
    tree = np.vstack((tree,np.zeros((2,4))))
    next = tree.shape[0]
    tree[cur,2]=next-2
    tree[cur,3]=next-1# заполнили информацию о потомках в текущую ветку
    tree[next-2,0]=-1
    tree[next-1,0]=-1
    tree[next-2,1]=maxProb(l.iloc[:,0])#записали в потомках информацию на случай
    tree[next-1,1]=maxProb(r.iloc[:,0])# если они останутся листами
    tree1 = makeTree(next-2,tree,deep+1,l,listCount+1,numParts,stopcrit)
    tree2 = makeTree(next-1,tree1,deep+1,r,listCount+1,numParts,stopcrit)

    if (tree1.shape[0]>tree2.shape[0]):
      tree = tree1.copy()
      for i in range(tree2.shape[0]):
        if tree2[i,2]!=0:
          tree[i]=tree2[i]
    else:
      tree = tree2.copy()
      for i in range(tree1.shape[0]):
        if tree1[i,2]!=0:
          tree[i]=tree1[i]
# обновили информацию по поводу дерева
  return tree

[номер признака, значение признака, левый потомок, правый потомок]

In [ ]:
def pred(tree, x):
  y = np.zeros(x.shape[0])
  j=0
  for i in x.values:
    treeI=0
    while tree[treeI,0]!=-1:
      if i[int(tree[treeI,0])-1]>tree[treeI,1]:
        treeI=tree[treeI,3]
      else:
        treeI=tree[treeI,2]
      treeI=int(treeI)
    y[j]=tree[treeI,1]
    j+=1
  return y

In [ ]:
# критерий остановки задаем массивом где 1 это использовать критерий, 0 - не использовать, чтобы мы могли комбинировать их
tree = np.zeros((1,4))
tree = makeTree(0,tree,1,x_train,1,30,[0,0,0]) #критерий остановки только пьюрити
p = pred(tree,x_test)
print(accuracy_score(y_test,p))
print(tree)

tree = np.zeros((1,4))
tree = makeTree(0,tree,1,x_train,1,30,[0,0,1])#останавливаемся при глубине = 5
p = pred(tree,x_test)
print(accuracy_score(y_test,p))
print(tree)

tree = np.zeros((1,4))
tree = makeTree(0,tree,1,x_train,1,30,[0,1,0])#останавливаемся при колличестве элементов в листе 5 или меньше
p = pred(tree,x_test)
print(accuracy_score(y_test,p))
print(tree)

tree = np.zeros((1,4))
tree = makeTree(0,tree,1,x_train,1,30,[1,0,0])#останавливаемся при колличестве листьев = 5
p = pred(tree,x_test)
print(accuracy_score(y_test,p))
print(tree)

In [ ]:
plt.scatter(df.iloc[:50,3],df.iloc[:50,4])# визуализация данных
plt.scatter(df.iloc[50:100,3],df.iloc[50:100,4])
plt.scatter(df.iloc[100:150,3],df.iloc[100:150,4])

С каждым из критериев остановки точность дерева больше, чем только с purity, но сами резудьтаты вышли одинаковыми, что связано с небольшим колличеством данных, и довольно схожими начальными разделениями, но можно заметить что дял остановки по глубине или по колличеству листьев, при той же точности, само дерево короче


##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес".

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
!gdown 1F5Vh-TTLqD9lqJ9deHlXJKIkyvTuE5wR

In [ ]:
from numpy.lib.function_base import append
df = pd.read_csv(filepath_or_buffer = "churn.csv")
df = df.drop(['RowNumber','CustomerId','Surname'], axis=1)#удаляю так как сама инструкция к датабазе выделяет эти поля как неважные
df.Gender = np.where(df.Gender == 'Male', 0, df.Gender)
df.Gender = np.where(df.Gender == 'Female', 1, df.Gender)
df['isFrance']=df.Geography.apply(lambda x: 1 if x=="France" else 0)
df['isSpain']=df.Geography.apply(lambda x: 1 if x=="Spain" else 0)
df['isGermany']=df.Geography.apply(lambda x: 1 if x=="Germany" else 0)
df=df.drop(columns = ['Geography'],axis = 1)
df

In [ ]:
data=df.drop(['Exited'], axis=1)
target=df.Exited.to_frame()
data

from sklearn.utils import shuffle
from sklearn import tree
from sklearn.model_selection import train_test_split

def makeData(x,y):
  l=len(y)
  x['Exited']=y
  x=shuffle(x)         #если мы каждый раз будем полностью рандомить все строчки, для построения леса у нас будет уходить слишком много времени
  proc = round(l*0.3)  #а нам понадобится его строить помногу раз чтобы подобрать лучшие гиперпараметры
  x = x.iloc[proc:,:]  #поэтому мы перемешиваем строки, удаляем примерно треть, и копируем столькоже
  x = pd.concat([x.iloc[proc:, :], x.iloc[0:proc, :]], ignore_index=True) # это примерно то же самое что рандомно брать все строки
  rx=x.drop(['Exited'], axis=1)
  ry=x.Exited.to_frame()
  return rx,ry

def trainForest(x,y,numTrees, min_samples_leaf=1, depth=None, min_impurity_decrease=0):
  clf=[]
  for i in range(numTrees):
    clf.append(tree.DecisionTreeClassifier(max_depth = depth,\
                                           max_features = 'sqrt',\
                                           min_samples_leaf=min_samples_leaf,\
                                           min_impurity_decrease=min_impurity_decrease))
    rx,ry=makeData(x,y)
    clf[i]=clf[i].fit(rx,ry)
  return clf

def predict(trees, x):
  y=np.zeros(len(x))
  for i in trees:
    y=y+i.predict(x)
  y=y//((len(trees)//2)+1)
  return y

x_train, x_test, y_train, y_test = train_test_split(data, target)

maxi = 0
n,m,d,i=0,0,0,0
minSamp = 1

In [ ]:
#здесь мы перебираем деревья чтобы найти лучшее(закоментировано, так как заняло много времени)

# for numTree in range(10,101,10):
#     print(numTree, minSamp)
#     print(maxi,n,m,d,i)
#     for dep in range(5,51,5):
#       x = np.linspace(0.0001,0.001,10)
#       for imp in x:
#         clf = trainForest(x_train,y_train,numTree,minSamp,dep,imp)
#         proc = (predict(clf,x_test).reshape(1,-1) == y_test.values.reshape(1,-1))
#         if(sum(sum(proc))/2500*100>maxi):
#           maxi=sum(sum(proc))/2500*100
#           n=numTree
#           m=minSamp
#           d=dep
#           i=imp

# print(n,m,d,i)

print(30, 1, 10, 0.0001) #результат и лучшие пораметры судя по перебору

по итогу перебора вышло: \\
лучшее колличество деревьев - 30 \\
минимальное количество листьев в дереве - 1 \\
максимальная глубина дерева - 10 \\
минимальное изменение функционала качества 0.0001

In [ ]:
mi=100
ma=0
for i in range (10):
  t = trainForest(x_train,y_train,30, 1, 10, 0.0001)
  proc = (predict(t,x_test).reshape(1,-1) == y_test.values.reshape(1,-1))
  mi=min(mi,sum(sum(proc))/2500*100)
  if (ma<sum(sum(proc))/2500*100):
    ma=sum(sum(proc))/2500*100
    bestTree = t
 #худший и лучший результат леса с данными параметрами за 10 прогонов
mi,ma

In [ ]:
bestTree[0].feature_importances_# показывает важность фич в дереве, найдем среднее по всем деревьям в лесу

topF=np.zeros_like(bestTree[0].feature_importances_)
l=len(bestTree)
for i in range(l):
  topF+=bestTree[i].feature_importances_
print(topF/l)

attributes = pd.DataFrame(data = {'Attribute': data.columns,'Importance' : topF/l}).sort_values(by = 'Importance', ascending = False).head()
attributes

Мы видем что самым важным пораметром оказался второй(с нуля), что не удевительно, так как это возраст клиента, так же к сильно значемым пораметрам можно отнести 5 и 4 (в этом порядке), а это количество продуктов купленных у банка и баланс соответственно, по большей части именно эти три категории дают нам понять останется ли клиент у банка. Соответственно небольшой возраст, отсутствие купленых продуктов банка и небольшой баланс в банке является индекатором того, что клиент скорее всего покинет банк.